In [1]:
import os
import pickle
from tqdm import tqdm
import numpy as np
import torch
from matplotlib import pyplot as plt
from torch import nn, optim
from torch.utils.data import DataLoader
from datasets.ascad import AscadDataset
from models.effcnn import EffCnn_N0
from datasets.transforms import ToTensor, ToLabelTensor
from training.training import execute_epoch, train_batch, eval_batch
from training.metrics import get_loss, get_accuracy, get_mean_rank, get_rank_over_time

In [2]:
num_epochs = 100
batch_size = 256
batch_metrics = {'loss': get_loss, 'acc': get_accuracy, 'mnrk': get_mean_rank}
epoch_metrics = {'rot': get_rank_over_time}
results_dir = os.path.join('.', 'results', 'cnn_classifier')
retrain = True

In [3]:
train_dataset = AscadDataset(train=True, transform=ToTensor(), target_transform=ToLabelTensor())
test_dataset = AscadDataset(train=False, transform=ToTensor(), target_transform=ToLabelTensor())
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
model = EffCnn_N0(train_dataset.input_shape)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
device = 'cuda'
model = model.to(device)
print('Model:', model)
print('Loss function:', loss_fn)
print('Optimizer:', optimizer)
print('Device:', device)

Model: EffCnn_N0(
  (feature_extractor): Sequential(
    (0): Conv1d(1, 4, kernel_size=(1,), stride=(1,))
    (1): SELU()
    (2): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  )
  (fc): Sequential(
    (0): Linear(in_features=1400, out_features=10, bias=True)
    (1): SELU()
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): SELU()
    (4): Linear(in_features=10, out_features=256, bias=True)
  )
)
Loss function: CrossEntropyLoss()
Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)
Device: cuda


In [5]:
train_results = {}
test_results = {}

def train_epoch(update_params=True):
    if update_params:
        results = execute_epoch(train_batch, train_dataloader, model, loss_fn, optimizer, device,
                                batch_metric_fns=batch_metrics, epoch_metric_fns=epoch_metrics)
    else:
        results = execute_epoch(eval_batch, train_dataloader, model, loss_fn, device,
                                batch_metric_fns=batch_metrics, epoch_metric_fns=epoch_metrics)
    for key in results.keys():
        if not(key in train_results.keys()):
            train_results[key] = []
        train_results[key].append(results[key])
def test_epoch():
    results = execute_epoch(eval_batch, test_dataloader, model, loss_fn, device,
                            batch_metric_fns=batch_metrics, epoch_metric_fns=epoch_metrics)
    for key in results.keys():
        if not(key in test_results.keys()):
            test_results[key] = []
        test_results[key].append(results[key])

if retrain:
    train_epoch(update_params=False)
    test_epoch()
    for epoch in tqdm(range(num_epochs)):
        train_epoch()
        test_epoch()

  6%|██████████▏                                                                                                                                                              | 6/100 [00:26<06:49,  4.36s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/min/a/jgammell/anaconda3/envs/sca_defense/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_14733/209427119.py", line 28, in <module>
    test_epoch()
  File "/tmp/ipykernel_14733/209427119.py", line 16, in test_epoch
    results = execute_epoch(eval_batch, test_dataloader, model, loss_fn, device,
  File "/local/scratch/a/jgammell/sca_defense/training/training.py", line 6, in execute_epoch
    for batch in dataloader:
  File "/home/min/a/jgammell/anaconda3/envs/sca_defense/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 517, in __next__
    data = self._next_data()
  File "/home/min/a/jgammell/anaconda3/envs/sca_defense/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 557, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
  File "/home/min/a/jgammell/anaco

TypeError: object of type 'NoneType' has no len()

In [ ]:
if retrain:
    os.makedirs(results_dir, exist_ok=True)
    with open(os.path.join(results_dir, 'train_results.pickle'), 'wb') as F:
        pickle.dump(train_results, F)
    with open(os.path.join(results_dir, 'test_results.pickle'), 'wb') as F:
        pickle.dump(test_results, F)

In [ ]:
with open(os.path.join(results_dir, 'train_results.pickle'), 'rb') as F:
    train_results = pickle.load(F)
with open(os.path.join(results_dir, 'test_results.pickle'), 'rb') as F:
    test_results = pickle.load(F)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
train_loss = [np.mean(x) for x in train_results['loss']]
test_loss = [np.mean(x) for x in test_results['loss']]
train_acc = [np.mean(x) for x in train_results['acc']]
test_acc = [np.mean(x) for x in test_results['acc']]
train_mnrk = [np.mean(x) for x in train_results['mnrk']]
test_mnrk = [np.mean(x) for x in test_results['mnrk']]
axes[0].plot(train_loss, '--', color='blue')
axes[0].plot(test_loss, '-', color='blue')
axes[1].plot(train_acc, '--', color='blue')
axes[1].plot(test_acc, '-', color='blue')
axes[2].plot(train_mnrk, '--', color='blue')
axes[2].plot(test_mnrk, '-', color='blue')
axes[0].set_title('Loss')
axes[1].set_title('Accuracy')
axes[2].set_title('Mean rank')
axes[0].set_xlabel('Epoch')
axes[1].set_xlabel('Epoch')
axes[2].set_xlabel('Epoch')
axes[0].set_ylabel('Value')
axes[1].set_ylabel('Value')
axes[2].set_ylabel('Value')
plt.tight_layout()